# 発端
kaggle「Microsoft Malware Prediction」にて、水準数（値の種類）が多い変数の扱いに苦慮。  
これの対処をどのようにすべきかkenelを見ていたところ、  
以下に記載されていたFrequency encodingについて未知だったので調べてメモしたもの。  
https://www.kaggle.com/fabiendaniel/detecting-malwares-with-lgbm  
いい機会なのでカテゴリ変数のencoding手法について纏める。

# 参照ページ
公式ページは各手法欄に記載  
https://mikebird28.hatenablog.jp/entry/2018/05/19/213047

# カテゴリ変数のエンコーディングの意義  
多くの機械学習のモデルでは、カテゴリ変数をそのままモデルに投入し学習させることができない。  
そのため、カテゴリ変数の値を数値化する必要がある。これがエンコーディング。  
エンコーディングをして値を数値化することで初めて学習をさせることが可能になる。  

# 手法
## sklearnを使う
### Label Encoder  
* sklearnの公式ページ  
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html  
存在する値を0〜値の種類の数-1の間の整数に割り当てる  
カテゴリ変数を数値化するだけでなく、数値変数の値を正規化する用途でも使うことができる。  
（もしかしたら後者の用途がメインなのかも）  
エンコード前の値が数値の場合は昇順で数値化され、カテゴリの場合は文字コード順のような気がする。

**所感**  
値間に上下、大小といった優劣の関係があるものに使用したほうが良いような気がする。  
単純に種類（犬好き、ネコ好きのような上下がないラベル）の場合は回帰タスクでは避けたほうが良さそう。  
規則化していない値が出現した場合はエラーになってしまうので留意する必要がある。

#### 数値変数の標準化  
数値の値同士の間隔を同等にする。
「どれくらい」の情報を削除して大小だけの情報にする。

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df = pd.DataFrame({'category':[10,5,200,3000,5],})
le = LabelEncoder()
df['encoded_value'] = le.fit_transform(df['category'])
df

,category,encoded_value
0,10,1
1,5,0
2,200,2
3,3000,3
4,5,0


#### カテゴリ変数の数値化  
文字で表現されている値を数値化して表現する。

In [4]:
#カテゴリ変数の標準化
#水準が英語の場合はA,B,C順に数値になっていそう
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df = pd.DataFrame({
    'category' : ['Dog','Essence','China','Bag'],
})

le = LabelEncoder()
df['encoded_feature'] = le.fit_transform(df['category'])
df

,category,encoded_feature
0,Dog,2
1,Essence,3
2,China,1
3,Bag,0


In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df1 = pd.DataFrame({'category':['スーパードライ','エビス','プレモル','一番搾り']})
le = LabelEncoder()
df1['encoded_label'] = le.fit_transform(df1['category'])
df1

,category,encoded_label
0,スーパードライ,1
1,エビス,0
2,プレモル,2
3,一番搾り,3


In [3]:
#df2 = pd.DataFrame({'category':['99.99(フォーナイン)','エビス','プレモル','一番搾り']})
df2 = pd.DataFrame({'category':['エビス','プレモル','一番搾り']})
df2['encoded_label'] = le.transform(df2['category'])
df2

,category,encoded_label
0,エビス,0
1,プレモル,2
2,一番搾り,3


### Ordinal Encoding
* 公式ページ  
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html#sklearn.preprocessing.OrdinalEncoder  
配列型のインプットを序数で表現する  

**所感**  
LabelEncoding同様回帰では使えないという認識。


In [1]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
X = [['Male', 1,'a'], ['Female', 3,'a'], ['Nonmale', 2,'b'],['Female',10,'c']]

X_enc = enc.fit_transform(X)
X_enc

array([[1., 0., 0.],
       [0., 2., 0.],
       [2., 1., 1.],
       [0., 3., 2.]])

In [38]:
X

[['Male', 1, 'a'],
 ['Female', 3, 'a'],
 ['Nonmale', 2, 'b'],
 ['Female', 10, 'c']]

In [13]:
#Xの値で作成したルールをYに適用させる
Y = [['Female',10,'c'], ['Male',2,'b'], ['Male',2,'a'], ['Female',3,'c']]
Y_enc = enc.transform(Y)
Y_enc

array([[0., 3., 2.],
       [1., 1., 1.],
       [1., 1., 0.],
       [0., 2., 2.]])

In [11]:
Y

[['Female', 10, 'c'], ['Male', 2, 'b'], ['Male', 2, 'a'], ['Female', 3, 'c']]

### One Hot Encoding  
* 公式ページ  
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder  

カテゴリをひとつづつカラムとして、1or0で値を表現する。  
多重共線性（マルチコ）を防ぐため、一つはカラムを削除することが多い。  
（例：男女というカテゴリを持つカラムでOne Hot Encodingを使用する場合は、男カラムのみを残し、女カラムを削除する）
男フラグが1ならば男であり、0ならば女と判断可能なため。  
2つのカラムを残しておくと多重共線性によって正しく推計できなくなるため。  
水準数が多い場合は向かなそう。  

In [1]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore')

X = [['竹鶴',100],['ニッカ',20],['山崎',500],['響',1000]]
enc_X = encoder.fit_transform(X)
enc_X.toarray()

array([[0., 0., 1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 1.]])

In [2]:
encoder.get_feature_names()

array(['x0_ニッカ', 'x0_山崎', 'x0_竹鶴', 'x0_響', 'x1_20', 'x1_100', 'x1_500',
       'x1_1000'], dtype=object)

In [6]:
import pandas as pd
df = pd.DataFrame(enc_X.toarray(),columns=encoder.get_feature_names())
df

,x0_ニッカ,x0_山崎,x0_竹鶴,x0_響,x1_20,x1_100,x1_500,x1_1000
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


## category_encodersを使う  
http://contrib.scikit-learn.org/categorical-encoding/index.html  

sklearnを利用したエンコードライブラリ。  
未知のクラスや欠損値は0を入れてくれる。  

### OrdinalEncoder

In [8]:
import category_encoders as ce

encoder = ce.OrdinalEncoder()
X = ['スーパードライ','エビス','プレモル','一番搾り']
enc_X = encoder.fit_transform(X)

In [9]:
df1 = pd.DataFrame(X,columns = ['category'])
df1['encoded_label'] = enc_X
df1

,category,encoded_label
0,スーパードライ,1
1,エビス,2
2,プレモル,3
3,一番搾り,4


In [10]:
X2 = ['99.99(フォーナイン)','一番搾り','プレモル','よなよな','一番搾り']
enc_X2 = encoder.transform(X2)
df2 = pd.DataFrame(X2,columns = ['category'])
df2['encoded_label'] = enc_X2
df2['inverse_label'] = encoder.inverse_transform(enc_X2)
df2

,category,encoded_label,inverse_label
0,99.99(フォーナイン),0,None
1,一番搾り,4,一番搾り
2,プレモル,3,プレモル
3,よなよな,0,None
4,一番搾り,4,一番搾り


In [5]:
import pandas as pd
import numpy as np
import category_encoders as ce

df_test = pd.DataFrame({'col1':['aaa','bbb','ccc','ccc'],
          'beer':['スーパードライ','よなよな','エビス','よなよな'],
                       'whiskey':['山崎','トリス','ニッカ','トリス'],
                       '数字':[100,20,30,200]})
df_test

,col1,beer,whiskey,数字
0,aaa,スーパードライ,山崎,100
1,bbb,よなよな,トリス,20
2,ccc,エビス,ニッカ,30
3,ccc,よなよな,トリス,200


In [6]:
encoder2 = ce.OrdinalEncoder()
enc_test = encoder2.fit_transform(df_test)
enc_test

,col1,beer,whiskey,数字
0,1,1,1,100
1,2,2,2,20
2,3,3,3,30
3,3,2,2,200


In [9]:
df_test2 = pd.DataFrame({'col1':['ddd','bbb','ccc','ccc',np.nan],
          'beer':['水曜日のねこ','よなよな','エビス','黒ラベル','クリアアサヒ'],
                       'whiskey':[np.nan,'ホワイトホース','ニッカ','トリス','紹興酒'],
                        '数字':[300,1000,400,200000,10]})
df_test2

,col1,beer,whiskey,数字
0,ddd,水曜日のねこ,NaN,300
1,bbb,よなよな,ホワイトホース,1000
2,ccc,エビス,ニッカ,400
3,ccc,黒ラベル,トリス,200000
4,NaN,クリアアサヒ,紹興酒,10


In [10]:
enc_test2 = encoder2.transform(df_test2)
enc_test2

,col1,beer,whiskey,数字
0,0,0,0,300
1,2,2,0,1000
2,3,3,3,400
3,3,0,2,200000
4,0,0,0,10


In [12]:
encoder2.fit(df_test)

OrdinalEncoder(cols=['col1', 'beer', 'whiskey'], drop_invariant=False,
        handle_unknown='impute', impute_missing=True,
        mapping=[{'col': 'col1', 'mapping': [('aaa', 1), ('bbb', 2), ('ccc', 3)]}, {'col': 'beer', 'mapping': [('スーパードライ', 1), ('よなよな', 2), ('エビス', 3)]}, {'col': 'whiskey', 'mapping': [('山崎', 1), ('トリス', 2), ('ニッカ', 3)]}],
        return_df=True, verbose=0)

## libraryを用いない  
### Freaquency Encoding  
各値に対して、その値の出現頻度を全体の行数で割った値を与える。

In [10]:
import pandas as pd

df = pd.DataFrame({
    'category':['せせり','せせり','せせり','豚タン','豚タン','ししとう','ネギま'],
    'quantity':[10,20,11,4,10,15,4],
    'flg':[1,1,0,0,0,1,1]
})

#各カテゴリの値の出現回数をカウントする
grouped = df.groupby('category').size().reset_index(name = 'category_counts')

#dfに出現回数のカラムを追加する
df = df.merge(grouped, how = 'left',on = 'category')

#出現割合列を追加
df['freq'] = df['category_counts']/df['category_counts'].count()

df


,category,quantity,flg,category_counts,freq
0,せせり,10,1,3,0.428571
1,せせり,20,1,3,0.428571
2,せせり,11,0,3,0.428571
3,豚タン,4,0,2,0.285714
4,豚タン,10,0,2,0.285714
5,ししとう,15,1,1,0.142857
6,ネギま,4,1,1,0.142857


### Target Mean Encoding  
Likelihood Encodingとも呼ばれる。  
カテゴリ変数のうち、目的変数が1の個数をその出現回数で割ったもの。

In [18]:
import pandas as pd

df = pd.DataFrame({
    'category':['せせり','せせり','せせり','豚タン','豚タン','ししとう','ネギま'],
    'quantity':[10,20,11,4,10,15,4],
    'flg':[1,1,0,0,0,1,1]
})


#カテゴリの出現回数と目的変数が1の行のカウント
grouped_category = df.groupby('category')['category'].count().reset_index(name = 'category_counts')
grouped_flg = df.groupby('category')['flg'].sum().reset_index(name = 'flg_counts')

#もとのDataFrameに結合
df = df.merge(grouped_category, how = 'left' ,on = 'category')
df = df.merge(grouped_flg, how = 'left' ,on = 'category')

df['taeget_mean_encoding'] = df['flg_counts']/df['category_counts']

df

,category,quantity,flg,category_counts,flg_counts,taeget_mean_encoding
0,せせり,10,1,3,2,0.666667
1,せせり,20,1,3,2,0.666667
2,せせり,11,0,3,2,0.666667
3,豚タン,4,0,2,0,0.000000
4,豚タン,10,0,2,0,0.000000
5,ししとう,15,1,1,1,1.000000
6,ネギま,4,1,1,1,1.000000


Traget Mean Encodingから過学習を防ぐために**leave one out schema**という手法も提案されている。  
これは、Target Mean Encodingを計算する際に自分自身を除く。  


In [21]:
import pandas as pd

df = pd.DataFrame({
    'category':['せせり','せせり','せせり','豚タン','豚タン','ししとう','ネギま'],
    'quantity':[10,20,11,4,10,15,4],
    'flg':[1,1,0,0,0,1,1]
})


#カテゴリの出現回数と目的変数が1の行のカウント
grouped_category = df.groupby('category')['category'].count().reset_index(name = 'category_counts')
grouped_flg = df.groupby('category')['flg'].sum().reset_index(name = 'flg_counts')

#もとのDataFrameに結合
df = df.merge(grouped_category, how = 'left' ,on = 'category')
df = df.merge(grouped_flg, how = 'left' ,on = 'category')

df['taeget_mean_encoding'] = (df['flg_counts'] - df['flg'])/(df['category_counts']-1)

df.fillna({'taeget_mean_encoding':0})

,category,quantity,flg,category_counts,flg_counts,taeget_mean_encoding
0,せせり,10,1,3,2,0.5
1,せせり,20,1,3,2,0.5
2,せせり,11,0,3,2,1.0
3,豚タン,4,0,2,0,0.0
4,豚タン,10,0,2,0,0.0
5,ししとう,15,1,1,1,0.0
6,ネギま,4,1,1,1,0.0
